In [1]:
from sae_core.analysis import SAEAnalyzer
from sae_core.pretrained import PRETRAINED_SAES, list_pretrained, load_pretrained
from sae_core.data_processing.textbook_process import load_processed_data
from transformer_lens import HookedTransformer
import numpy as np

/Users/deancasey/Documents/Columbia/CRIS Lab/SAELens/cris_py312_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


all the way through


In [2]:
qwen3_06B_l14_mlp_file_path = list_pretrained()[1]
qwen3_06B_14_mlp_SAE = load_pretrained(qwen3_06B_l14_mlp_file_path)

In [3]:
qwen3_06B_l14_mlp_file_path[-19:-17]

'14'

In [4]:
qwen3_06B = HookedTransformer.from_pretrained("qwen3-0.6b")

`torch_dtype` is deprecated! Use `dtype` instead!


Loaded pretrained model qwen3-0.6b into HookedTransformer


In [5]:
chemistry_texts = load_processed_data('sae_core/data/processed_data/processed_chem_text.json')[55:64]

In [6]:
qwen3_analyzer = SAEAnalyzer(qwen3_06B, qwen3_06B_l14_mlp_file_path, chemistry_texts)

Model and SAE on device: mps:0


In [7]:
qwen3_analyzer.hook_point

'blocks.14.hook_mlp_out'

In [ ]:
# Run analyses
print("Computing sparsity metrics...")
sparsity_results, features = qwen3_analyzer.compute_sparsity_metrics(chemistry_texts)

print("Finding dead features...")
dead_features = qwen3_analyzer.find_dead_features(features)

print("Computing reconstruction metrics...")
reconstruction = qwen3_analyzer.compute_layer_reconstruction_metrics(chemistry_texts)

print("Running ablation study...")
ablation = qwen3_analyzer.ablation_study(chemistry_texts)

# Print results
print("\n=== SPARSITY ===")
print(f"L0 Mean (# of Active Features): {sparsity_results['l0_mean']:.2f} out of 4096")
print(f"Dead Features: {dead_features['n_dead']} ({dead_features['pct_dead']:.1f}%)")

print("\n=== RECONSTRUCTION ===")
print(f"MSE Loss: {reconstruction['mse_loss']:.4f}")
print(f"Explained Variance: {reconstruction['explained_variance']:.3f}")

print("\n=== ABLATION ===")
print(f"Baseline Loss: {ablation['baseline_loss']:.4f}")
print(f"Zero Ablation Loss: {ablation['zero_ablation_loss']:.4f}")
print(f"SAE Reconstruction Loss: {ablation['sae_reconstruction_loss']:.4f}")
print(f"Loss Recovered: {ablation['loss_recovered']:.4f}")

# Find max activating examples for feature 0
print("\n=== TOP ACTIVATIONS FOR FEATURE 100 ===")
examples = qwen3_analyzer.find_max_activating_examples(chemistry_texts[:-5], feature_idx=100)
for i, ex in enumerate(examples[:5]):
    print(f"{i+1}. [{ex['activation']:.3f}] {ex['text']}")

Computing sparsity metrics...


Finding dead features...
Computing reconstruction metrics...
Running ablation study...
Original acts norm: 284.9996
Reconstructed norm: 800.9756
Difference norm: 735.5618
Original acts norm: 319.8231
Reconstructed norm: 854.9485
Difference norm: 781.0010
Original acts norm: 156.9252
Reconstructed norm: 410.6422
Difference norm: 374.7352

=== SPARSITY ===
L0 Mean (# of Active Features): 2113.27
Dead Features: 0 (0.0%)

=== RECONSTRUCTION ===
MSE Loss: 7.7865
Explained Variance: -5.060

=== ABLATION ===
Baseline Loss: 16.3338
Zero Ablation Loss: 16.6000
SAE Reconstruction Loss: 15.7980
Loss Recovered: 3.0130

=== TOP ACTIVATIONS FOR FEATURE 100 ===
1. [2.602] XXV. THE MAGNESIUM FAMILY
2. [2.454] XXVII. THE IRON FAMILY                                            
3. [2.046]  FAMILY                                  300
4. [2.044] XXIV. THE ALKALINE-E
5. [1.894]  THE IRON FAMILY                                            338<|endoftext|><|endoftext|>


In [17]:
# Check:
print(f"Encoder bias mean: {qwen3_analyzer.sae.b_enc.mean()}")
print(f"Decoder bias norm: {qwen3_analyzer.sae.b_dec.norm()}")

Encoder bias mean: -0.007955756038427353
Decoder bias norm: 0.12984265387058258


In [9]:
ablation

{'baseline_loss': 16.33381716410319,
 'zero_ablation_loss': 16.600012461344402,
 'sae_reconstruction_loss': 15.797977447509766,
 'loss_recovered': 3.012957111364287}

In [19]:
qwen3_06B.cfg.d_model

1024

In [10]:
qwen3_06B_14_mlp_SAE, qwen3_06B_14_mlp_SAE_history = load_pretrained(qwen3_06B_l14_mlp_file_path, load_history=True)

In [11]:
qwen3_06B_14_mlp_SAE_history.keys()

dict_keys(['loss', 'recon_loss', 'l1_loss', 'sparsity', 'total_post_layer_mse', '15_mse', '16_mse', '17_mse', '18_mse', '19_mse', '20_mse', '21_mse', '22_mse', '23_mse', '24_mse', '25_mse', '26_mse', '27_mse', 'logit_kl'])

In [12]:
# qwen3_06B_14_mlp_SAE_history['logit_kl']

In [13]:
# qwen3_06B_14_mlp_SAE_history['sparsity']

In [14]:
# qwen3_06B_14_mlp_SAE_history['loss']